In [5]:
def parseMGF(mgfData):
    data = mgfData.read_text().split('\n')
    _comments = '#;!/'
    reading_spectrum = False
    params = {}
    masses = []
    intensities = []
    charges = []
    out = {}
    cnt = 0
    pep_mass = 0
    pep_intensity = 0
    out = {}
    for line in data:
        if not reading_spectrum:
            if line.strip() == 'BEGIN IONS': reading_spectrum = True
        else:
            if not line.strip() or any(line.startswith(c) for c in _comments): pass
            elif line.strip() == 'END IONS':
                reading_spectrum = False
                title = params['title'].split()[0]
                if 'pepmass' in params:
                    try:
                        pl = params['pepmass'].split()
                        if len(pl) > 1:
                            pep_mass = float(pl[0])
                            pep_intensity = float(pl[1])
                        elif len(pl) == 1: pep_mass = float(pl[0])
                    except ValueError: print("Error in parsing pepmass value")
                out[cnt] = {'pep_mass': pep_mass,'pep_intensity': pep_intensity,'rtinseconds': params['rtinseconds'],'title': params['title'],'charge': params['charge'],'mz_array': np.array(masses),'intensity_array': np.array(intensities)}
                cnt += 1
            else:
                l = line.split('=', 1)
                if len(l) > 1: params[l[0].lower()] = l[1].strip()
                elif len(l) == 1:  # looks like a peak list ;)
                    l = line.split()
                    if len(l) >= 2000:
                        try:
                            masses.append(float(l[0]))
                            intensities.append(float(l[1]))
                        except ValueError:
                            print("Error in parsing line "+line)
    return out

In [6]:
import pathlib
import numpy as np

In [7]:
#file = pathlib.Path.cwd().parent.rglob('*.MGF')
#file = pathlib.Path.home()/'mgf' / '190128_robin_WT_5.raw.centroid.MGF'
file = pathlib.Path.home()/'mgf' / '190128_robin_ 6' / '190128_robin_WT_5.mgf'
print(file.exists())#read_text().split(' '))
out=parseMGF(file)

True


In [11]:
file_rr = pathlib.Path.home()/'mgf' / '190128_robin_WT_5.raw.centroid.MGF'
print(file_rr.exists())#read_text().split(' '))
out_rr=parseMGF(file_rr)

True


In [9]:
X=[(out[k]['pep_mass']-1.00727647)*int(out[k]['charge'].split('+')[0]) for k, _ in out.items()]
X=np.array(X).reshape(-1, 1)
print(X.shape)

(57697, 1)


In [12]:
X_rr=[(out_rr[k]['pep_mass']-1.00727647)*int(out_rr[k]['charge'].split('+')[0]) for k, _ in out_rr.items()]
X_rr=np.array(X_rr).reshape(-1, 1)
print(X_rr.shape)

(57969, 1)


In [10]:
import torch
print("#GPU-#", torch.cuda.device_count())

#GPU-# 1


In [28]:
for i in range(len(X)):
    if(X_rr[i]-X[i]>0.00001):
        print(i,out[i]['pep_mass'],out_rr[i]['pep_mass'])

5 452.87109 452.882019042969
8 469.32974 469.330169677734
10 476.94647 476.946563720703
19 388.34308 388.343231201172
20 447.34811 447.348114013672
21 388.89359 388.893737792969
22 388.34311 388.343139648438
24 452.87186 452.882171630859
26 380.89172 380.891754150391
30 388.3436 388.343658447266
32 447.34863 447.3486328125
33 448.35175 448.35205078125
34 447.34839 447.348510742188
35 536.16772 536.167724609375
38 447.34875 447.348754882813
42 358.9071 358.907318115234
47 447.34824 447.348266601563
53 434.88333 434.883392333984
60 376.90015 376.900207519531
66 376.90009 376.900177001953
76 447.34818 447.348205566406
78 447.3483 447.348327636719
84 448.35144 448.351501464844
91 454.23456 454.234588623047
96 491.25259 491.252990722656
97 528.27118 528.271423339844
98 536.16644 536.166809082031
100 484.74792 484.99853515625
103 448.35104 448.35107421875
104 491.25232 491.252960205078
105 456.95987 456.960418701172
107 499.99133 500.2421875
108 528.27051 528.271179199219
113 607.01007 607.2

1863 567.97406 568.307983398438
1872 555.27307 555.609985351563
1873 447.73895 447.738952636719
1881 600.28235 600.282653808594
1883 738.8465 738.847229003906
1891 393.88821 393.888214111328
1897 607.96033 608.294921875
1900 493.23114 493.48046875
1901 449.22397 449.474761962891
1902 598.63 598.630004882813
1910 897.44031 897.943664550781
1911 574.97754 574.977844238281
1912 807.36816 807.869140625
1913 741.68945 742.023742675781
1918 518.89941 518.8994140625
1920 648.30121 648.634704589844
1921 362.19363 362.193634033203
1924 956.94153 957.443054199219
1927 459.99576 460.234649658203
1929 452.21368 452.213684082031
1931 464.71902 464.719024658203
1932 611.61194 611.945983886719
1940 685.63843 685.972229003906
1941 357.8447 357.844787597656
1945 381.22073 381.220733642578
1948 368.19662 368.196624755859
1953 924.88147 925.381286621094
1956 722.68127 723.35009765625
1958 605.27344 605.276733398438
1960 561.77362 562.026062011719
1962 748.69843 749.031799316406
1964 514.22943 514.4802246

3401 595.59814 595.598205566406
3405 350.5545 350.554504394531
3406 1018.12115 1018.45672607422
3409 850.39606 850.897766113281
3411 418.22888 418.479400634766
3413 669.29718 669.632629394531
3414 804.35638 804.607299804688
3420 440.57922 440.579223632813
3421 848.05701 848.393371582031
3424 907.64215 908.143676757813
3427 1193.85132 1194.18408203125
3428 1140.16687 1140.5029296875
3430 1086.47632 1086.81066894531
3431 546.61469 546.614929199219
3433 1072.13867 1072.47375488281
3437 800.60876 800.608764648438
3442 917.89807 918.148803710938
3443 891.89514 892.14306640625
3446 576.59131 576.591491699219
3447 712.87201 712.87255859375
3448 450.25031 450.250549316406
3450 786.10242 786.352783203125
3457 604.31854 604.568237304688
3458 1047.80078 1048.13452148438
3459 932.15369 932.404846191406
3460 1156.16919 1156.50354003906
3464 790.10077 790.351257324219
3465 1174.84265 1175.17700195313
3468 586.80688 587.057678222656
3471 734.85474 734.857788085938
3474 1223.53052 1224.19750976563
347

4648 350.21198 350.212005615234
4649 749.89124 749.8916015625
4650 686.35492 686.844909667969
4653 921.97015 922.474914550781
4655 888.92981 889.431274414063
4657 835.15259 835.653381347656
4663 712.87225 712.872741699219
4664 786.90991 786.909973144531
4670 513.22571 513.4765625
4671 351.87411 351.874114990234
4678 647.81982 647.819946289063
4679 522.2641 522.264221191406
4690 852.61865 852.869750976563
4694 907.63654 907.637084960938
4702 483.25763 483.508544921875
4705 530.29504 530.295043945313
4709 461.74176 461.741882324219
4710 893.13177 893.633544921875
4720 500.24368 500.494445800781
4725 929.64734 930.148498535156
4732 879.12732 879.3779296875
4734 1190.50647 1191.17639160156
4735 889.13367 889.635498046875
4738 601.81708 601.817260742188
4739 617.77094 618.8388671875
4740 1136.48889 1136.822265625
4750 1185.17834 1185.84240722656
4755 844.44385 844.443969726563
4764 944.40472 944.6552734375
4770 897.96484 897.96533203125
4783 675.85449 675.854736328125
4785 549.94312 549.943

6194 768.65692 768.991394042969
6195 941.19659 941.445861816406
6198 1055.04761 1055.548828125
6203 802.09009 802.757873535156
6208 901.46783 901.467834472656
6210 1135.25305 1135.58679199219
6211 370.13821 370.138305664063
6220 382.19662 382.196624755859
6222 547.93317 548.267639160156
6226 401.54855 401.548614501953
6227 601.81903 601.819213867188
6235 949.50464 950.005676269531
6238 912.48553 912.98779296875
6239 544.25433 544.254333496094
6240 789.42505 789.924865722656
6242 897.96558 897.965698242188
6250 917.48315 917.985900878906
6251 1152.48303 1152.98388671875
6255 645.30505 645.641052246094
6256 981.01117 981.512817382813
6258 797.89899 797.899475097656
6261 880.46497 880.967651367188
6268 553.77716 554.027770996094
6270 888.9541 889.460693359375
6271 373.21466 373.214691162109
6272 564.7876 564.787719726563
6273 487.25485 487.505279541016
6281 972.0047 972.504638671875
6286 741.34332 741.344116210938
6294 1046.99512 1047.49853515625
6297 484.24493 484.244934082031
6298 1134.

...

53083 565.16174 1491.86218261719
53084 1035.90771 1464.37243652344
53085 776.35876 1502.869140625
53086 689.81848 1475.88305664063
53089 822.3222 1436.86267089844
53090 636.65894 1464.61975097656
53091 666.40289 1514.13012695313
53092 575.3515 1448.62817382813
53093 761.68488 1486.88098144531
53095 716.43329 1453.11437988281
53097 734.41779 1470.38171386719
53098 880.79425 1486.63391113281
53099 561.06519 1524.8837890625
53100 1080.64502 1458.87390136719
53102 685.51453 910.542907714844
53103 1096.14551 1497.14526367188
53104 717.32214 1535.89025878906
53105 582.35181 894.536743164063
53106 659.3996 1481.13928222656
53107 861.37073 1508.64880371094
53108 847.88641 1519.66149902344
53110 619.57629 710.928405761719
53111 477.36288 699.426330566406
53112 795.70203 1547.14697265625
53114 568.14728 1492.14587402344
53115 1071.03003 1502.6240234375
53116 580.7533 779.469482421875
53117 737.22571 1558.40368652344
53118 786.97461 1096.17260742188
53120 615.37482 897.540283203125
53121 632.3641

53864 535.32501 692.417724609375
53865 539.28271 670.405883789063
53869 553.33368 597.846862792969
53873 547.33209 554.338317871094
53880 434.88187 749.450927734375
53881 495.83286 497.23486328125
53882 812.48914 862.517211914063
53883 582.88751 759.461730957031
53884 515.26947 556.6708984375
53886 600.1994 575.32666015625
53889 661.39783 829.834411621094
53891 670.40765 623.378234863281
53892 693.41968 695.417236328125
53898 647.34839 663.401794433594
53899 606.23993 648.391906738281
53900 563.3761 579.351867675781
53901 491.29852 556.588928222656
53903 543.33008 575.348205566406
53906 630.7077 834.838684082031
53908 569.31372 610.03515625
53910 539.28259 619.375244140625
53914 451.26627 598.529541015625
53916 603.32196 626.378967285156
53922 598.13092 767.4619140625
53923 562.3382 846.508544921875
53924 537.30353 544.330383300781
53925 607.87177 572.338134765625
53926 430.88782 667.174499511719
53929 626.38043 815.490417480469
53932 679.40961 690.410095214844
53934 714.58124 732.4437

54803 604.16193 607.368591308594
54806 495.83368 702.329895019531
54811 601.86816 850.888488769531
54813 582.29926 670.405029296875
54814 599.43768 601.859680175781
54815 643.58856 649.282287597656
54816 663.33679 707.168701171875
54819 667.17712 762.960998535156
54820 565.37189 619.19873046875
54822 639.40863 686.357238769531
54823 621.43475 688.912475585938
54826 603.35132 659.89208984375
54828 434.88266 693.30908203125
54830 553.33691 721.486206054688
54832 379.24533 577.336242675781
54834 451.26816 732.445007324219
54836 633.15057 673.32666015625
54837 628.25354 712.65478515625
54838 395.23303 670.79443359375
54839 362.69537 598.748168945313
54842 403.21902 755.455444335938
54843 582.85889 645.298828125
54844 531.41467 857.452331542969
54845 380.74039 597.361083984375
54847 590.84479 607.308044433594
54849 770.48486 655.024047851563
54850 372.75415 589.465515136719
54852 370.68207 714.4326171875
54857 424.75101 603.323303222656
54858 574.76422 643.294006347656
54863 789.46686 679.4

55799 474.29315 579.3525390625
55803 621.43555 790.466857910156
55805 491.29901 720.90673828125
55806 531.29303 623.950317382813
55807 709.45245 815.217590332031
55810 515.4248 605.38916015625
55811 566.88953 623.749084472656
55815 572.99127 649.363220214844
55816 533.34637 712.910217285156
55817 577.32178 601.338623046875
55818 498.9025 663.079406738281
55820 399.25378 583.347045898438
55823 430.91464 467.276580810547
55828 378.77603 399.289459228516
55830 581.23248 654.867431640625
55834 581.31964 621.435668945313
55835 718.90784 760.437072753906
55838 579.59137 608.799438476563
55839 768.99207 751.390625
55840 529.35394 646.810363769531
55843 475.34644 529.353149414063
55844 676.43182 689.494201660156
55849 523.32532 647.36328125
55850 551.35657 718.816528320313
55851 599.45361 607.871337890625
55859 672.27985 763.608276367188
55862 654.37244 711.346130371094
55863 673.45459 713.40380859375
55865 597.60608 734.31591796875
55866 399.28931 529.314270019531
55867 456.28244 601.33691406

56783 543.10175 850.251586914063
56784 803.95972 628.196472167969
56785 525.28882 581.322692871094
56787 676.88177 545.404357910156
56788 587.32556 695.939880371094
56790 577.32227 763.434143066406
56791 569.31482 938.866577148438
56798 401.21448 575.342895507813
56799 667.44159 997.570739746094
56801 754.05182 826.808654785156
56805 812.09424 807.131408691406
56808 531.04291 567.424194335938
56810 562.12885 639.40625
56815 498.90195 576.396240234375
56816 509.30865 744.202453613281
56820 773.93909 681.449523925781
56821 675.47382 859.555847167969
56823 719.52301 627.499328613281
56829 515.4245 733.558410644531
56831 378.77573 541.335021972656
56833 595.61731 695.486633300781
56834 509.30865 523.324523925781
56836 561.1286 677.496337890625
56837 350.27655 599.361206054688
56840 579.82361 733.312805175781
56845 664.46448 698.401123046875
56848 573.13708 667.176879882813
56851 399.28961 653.034423828125
56852 523.41101 765.437744140625
56853 424.89764 649.360900878906
56855 430.91461 808

In [29]:
Device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(Device)

cuda:0


In [30]:
X_t=torch.tensor(X, dtype=torch.double, device=Device)
X_trr=torch.tensor(X_rr, dtype=torch.double, device=Device)

In [ ]:
import time
ts = time.time()
for i in range(len(X_t)):
    if(X_trr[i]-X_t[i]>0.00001):
        print(i,X_t[i],X_trr[i])
te = time.time()

5 tensor([903.7276], device='cuda:0', dtype=torch.float64) tensor([903.7495], device='cuda:0', dtype=torch.float64)
8 tensor([936.6449], device='cuda:0', dtype=torch.float64) tensor([936.6458], device='cuda:0', dtype=torch.float64)
10 tensor([951.8784], device='cuda:0', dtype=torch.float64) tensor([951.8786], device='cuda:0', dtype=torch.float64)
19 tensor([774.6716], device='cuda:0', dtype=torch.float64) tensor([774.6719], device='cuda:0', dtype=torch.float64)
20 tensor([1785.3633], device='cuda:0', dtype=torch.float64) tensor([1785.3634], device='cuda:0', dtype=torch.float64)
21 tensor([775.7726], device='cuda:0', dtype=torch.float64) tensor([775.7729], device='cuda:0', dtype=torch.float64)
22 tensor([774.6717], device='cuda:0', dtype=torch.float64) tensor([774.6717], device='cuda:0', dtype=torch.float64)
24 tensor([903.7292], device='cuda:0', dtype=torch.float64) tensor([903.7498], device='cuda:0', dtype=torch.float64)
26 tensor([759.7689], device='cuda:0', dtype=torch.float64) tens

191 tensor([2459.1477], device='cuda:0', dtype=torch.float64) tensor([2461.1540], device='cuda:0', dtype=torch.float64)
193 tensor([1890.9045], device='cuda:0', dtype=torch.float64) tensor([1891.9057], device='cuda:0', dtype=torch.float64)
194 tensor([1754.6837], device='cuda:0', dtype=torch.float64) tensor([1756.6965], device='cuda:0', dtype=torch.float64)
196 tensor([2573.1862], device='cuda:0', dtype=torch.float64) tensor([2574.1931], device='cuda:0', dtype=torch.float64)
197 tensor([1950.8586], device='cuda:0', dtype=torch.float64) tensor([1951.8634], device='cuda:0', dtype=torch.float64)
198 tensor([1701.7121], device='cuda:0', dtype=torch.float64) tensor([1701.7130], device='cuda:0', dtype=torch.float64)
199 tensor([1938.9111], device='cuda:0', dtype=torch.float64) tensor([1940.8265], device='cuda:0', dtype=torch.float64)
200 tensor([1701.7113], device='cuda:0', dtype=torch.float64) tensor([1701.7121], device='cuda:0', dtype=torch.float64)
201 tensor([1574.8371], device='cuda:0',

352 tensor([2995.2270], device='cuda:0', dtype=torch.float64) tensor([2996.2294], device='cuda:0', dtype=torch.float64)
353 tensor([1710.9582], device='cuda:0', dtype=torch.float64) tensor([1711.9601], device='cuda:0', dtype=torch.float64)
355 tensor([1816.8145], device='cuda:0', dtype=torch.float64) tensor([1816.8171], device='cuda:0', dtype=torch.float64)
356 tensor([1506.7517], device='cuda:0', dtype=torch.float64) tensor([1506.7544], device='cuda:0', dtype=torch.float64)
359 tensor([1537.8182], device='cuda:0', dtype=torch.float64) tensor([1538.8200], device='cuda:0', dtype=torch.float64)
361 tensor([1586.7116], device='cuda:0', dtype=torch.float64) tensor([1587.7145], device='cuda:0', dtype=torch.float64)
363 tensor([2207.9987], device='cuda:0', dtype=torch.float64) tensor([2210.0041], device='cuda:0', dtype=torch.float64)
364 tensor([1815.8138], device='cuda:0', dtype=torch.float64) tensor([1815.8138], device='cuda:0', dtype=torch.float64)
372 tensor([1666.6953], device='cuda:0',

537 tensor([1323.6917], device='cuda:0', dtype=torch.float64) tensor([1323.6917], device='cuda:0', dtype=torch.float64)
540 tensor([1963.9638], device='cuda:0', dtype=torch.float64) tensor([1964.9678], device='cuda:0', dtype=torch.float64)
542 tensor([1995.9298], device='cuda:0', dtype=torch.float64) tensor([1996.9292], device='cuda:0', dtype=torch.float64)
543 tensor([2036.9346], device='cuda:0', dtype=torch.float64) tensor([2037.9398], device='cuda:0', dtype=torch.float64)
545 tensor([1694.7075], device='cuda:0', dtype=torch.float64) tensor([1695.7152], device='cuda:0', dtype=torch.float64)
546 tensor([2057.6864], device='cuda:0', dtype=torch.float64) tensor([2058.6888], device='cuda:0', dtype=torch.float64)
548 tensor([1938.9057], device='cuda:0', dtype=torch.float64) tensor([1939.9091], device='cuda:0', dtype=torch.float64)
549 tensor([1938.9056], device='cuda:0', dtype=torch.float64) tensor([1939.9090], device='cuda:0', dtype=torch.float64)
553 tensor([2036.9329], device='cuda:0',

744 tensor([1874.8303], device='cuda:0', dtype=torch.float64) tensor([1874.8303], device='cuda:0', dtype=torch.float64)
745 tensor([1441.6823], device='cuda:0', dtype=torch.float64) tensor([1441.6823], device='cuda:0', dtype=torch.float64)
746 tensor([1906.9412], device='cuda:0', dtype=torch.float64) tensor([1907.9453], device='cuda:0', dtype=torch.float64)
747 tensor([1288.5555], device='cuda:0', dtype=torch.float64) tensor([1288.5559], device='cuda:0', dtype=torch.float64)
751 tensor([2050.8257], device='cuda:0', dtype=torch.float64) tensor([2050.8257], device='cuda:0', dtype=torch.float64)
755 tensor([2481.1450], device='cuda:0', dtype=torch.float64) tensor([2482.1520], device='cuda:0', dtype=torch.float64)
756 tensor([2093.9577], device='cuda:0', dtype=torch.float64) tensor([2094.9519], device='cuda:0', dtype=torch.float64)
758 tensor([2257.6849], device='cuda:0', dtype=torch.float64) tensor([2258.6917], device='cuda:0', dtype=torch.float64)
762 tensor([1664.7940], device='cuda:0',

...

24917 tensor([1276.6196], device='cuda:0', dtype=torch.float64) tensor([1455.8529], device='cuda:0', dtype=torch.float64)
24918 tensor([2370.2790], device='cuda:0', dtype=torch.float64) tensor([3867.8674], device='cuda:0', dtype=torch.float64)
24921 tensor([1111.5204], device='cuda:0', dtype=torch.float64) tensor([2058.0316], device='cuda:0', dtype=torch.float64)
24923 tensor([1790.8834], device='cuda:0', dtype=torch.float64) tensor([1845.9860], device='cuda:0', dtype=torch.float64)
24929 tensor([916.0285], device='cuda:0', dtype=torch.float64) tensor([2185.9400], device='cuda:0', dtype=torch.float64)
24931 tensor([1299.7132], device='cuda:0', dtype=torch.float64) tensor([1787.7475], device='cuda:0', dtype=torch.float64)
24932 tensor([1608.7777], device='cuda:0', dtype=torch.float64) tensor([1791.8824], device='cuda:0', dtype=torch.float64)
24933 tensor([1923.8863], device='cuda:0', dtype=torch.float64) tensor([2065.9612], device='cuda:0', dtype=torch.float64)
24936 tensor([2832.0838],

25086 tensor([2259.1352], device='cuda:0', dtype=torch.float64) tensor([2771.3942], device='cuda:0', dtype=torch.float64)
25090 tensor([1387.7960], device='cuda:0', dtype=torch.float64) tensor([1733.8399], device='cuda:0', dtype=torch.float64)
25091 tensor([1772.9679], device='cuda:0', dtype=torch.float64) tensor([1788.8751], device='cuda:0', dtype=torch.float64)
25095 tensor([996.5855], device='cuda:0', dtype=torch.float64) tensor([2260.1362], device='cuda:0', dtype=torch.float64)
25096 tensor([750.0072], device='cuda:0', dtype=torch.float64) tensor([2388.0066], device='cuda:0', dtype=torch.float64)
25097 tensor([1948.0592], device='cuda:0', dtype=torch.float64) tensor([3097.5429], device='cuda:0', dtype=torch.float64)
25098 tensor([1177.5924], device='cuda:0', dtype=torch.float64) tensor([1616.7529], device='cuda:0', dtype=torch.float64)
25099 tensor([967.5362], device='cuda:0', dtype=torch.float64) tensor([1387.7960], device='cuda:0', dtype=torch.float64)
25100 tensor([734.0360], de

25261 tensor([1975.1158], device='cuda:0', dtype=torch.float64) tensor([2162.0467], device='cuda:0', dtype=torch.float64)
25263 tensor([1416.6555], device='cuda:0', dtype=torch.float64) tensor([1807.8592], device='cuda:0', dtype=torch.float64)
25264 tensor([997.6306], device='cuda:0', dtype=torch.float64) tensor([2272.1255], device='cuda:0', dtype=torch.float64)
25266 tensor([1589.8394], device='cuda:0', dtype=torch.float64) tensor([2059.0363], device='cuda:0', dtype=torch.float64)
25270 tensor([1303.6568], device='cuda:0', dtype=torch.float64) tensor([1976.1189], device='cuda:0', dtype=torch.float64)
25271 tensor([2172.0783], device='cuda:0', dtype=torch.float64) tensor([2295.1611], device='cuda:0', dtype=torch.float64)
25276 tensor([1344.8162], device='cuda:0', dtype=torch.float64) tensor([2283.1265], device='cuda:0', dtype=torch.float64)
25277 tensor([1633.8632], device='cuda:0', dtype=torch.float64) tensor([2172.0765], device='cuda:0', dtype=torch.float64)
25278 tensor([981.6363], 

25430 tensor([1956.8834], device='cuda:0', dtype=torch.float64) tensor([2291.2373], device='cuda:0', dtype=torch.float64)
25434 tensor([1584.1643], device='cuda:0', dtype=torch.float64) tensor([2709.3972], device='cuda:0', dtype=torch.float64)
25438 tensor([1755.9609], device='cuda:0', dtype=torch.float64) tensor([2735.2615], device='cuda:0', dtype=torch.float64)
25440 tensor([1403.7346], device='cuda:0', dtype=torch.float64) tensor([3152.3654], device='cuda:0', dtype=torch.float64)
25441 tensor([1198.6745], device='cuda:0', dtype=torch.float64) tensor([2155.1808], device='cuda:0', dtype=torch.float64)
25442 tensor([2073.0776], device='cuda:0', dtype=torch.float64) tensor([3210.5971], device='cuda:0', dtype=torch.float64)
25448 tensor([1536.8012], device='cuda:0', dtype=torch.float64) tensor([2080.0191], device='cuda:0', dtype=torch.float64)
25452 tensor([2058.0333], device='cuda:0', dtype=torch.float64) tensor([2141.0297], device='cuda:0', dtype=torch.float64)
25453 tensor([2172.0777]

25609 tensor([1722.9066], device='cuda:0', dtype=torch.float64) tensor([2043.0215], device='cuda:0', dtype=torch.float64)
25610 tensor([1484.8985], device='cuda:0', dtype=torch.float64) tensor([3245.4296], device='cuda:0', dtype=torch.float64)
25611 tensor([1989.0003], device='cuda:0', dtype=torch.float64) tensor([2059.0401], device='cuda:0', dtype=torch.float64)
25612 tensor([1655.8453], device='cuda:0', dtype=torch.float64) tensor([1980.9571], device='cuda:0', dtype=torch.float64)
25613 tensor([1875.0067], device='cuda:0', dtype=torch.float64) tensor([2791.2735], device='cuda:0', dtype=torch.float64)
25620 tensor([1885.9339], device='cuda:0', dtype=torch.float64) tensor([1989.0003], device='cuda:0', dtype=torch.float64)
25621 tensor([1612.9358], device='cuda:0', dtype=torch.float64) tensor([1655.8453], device='cuda:0', dtype=torch.float64)
25622 tensor([1582.8376], device='cuda:0', dtype=torch.float64) tensor([1875.0067], device='cuda:0', dtype=torch.float64)
25623 tensor([2157.1437]

In [32]:
print(te - ts)

64.91320276260376


In [78]:
X_trr[:4]-X_t[:4]

tensor([[-6.8164e-06],
        [-8.4961e-06],
        [-3.5967e-04],
        [-1.1016e-04]], device='cuda:0', dtype=torch.float64)

In [79]:
print(torch.allclose(X_t[:4], X_trr[:4],atol=1e-04, rtol=1e-05,equal_nan=True))

True


In [83]:
print ((X_trr >= 774.6718).nonzero())

tensor([[    0,     0],
        [    1,     0],
        [    3,     0],
        ...,
        [57966,     0],
        [57967,     0],
        [57968,     0]], device='cuda:0')


In [14]:
partitions=45
iterations = 1
runs = 1
knnmode='random'

In [16]:
def knn_data(path):
    raw_data = sc.wholeTextFiles(path, use_unicode=False)
    mgf_data = process_data(raw_data).repartition(partitions).cache()
    parseMass = mgf_data.flatMap(lambda x: get_parsedata(x, 'pep_mass'))
    parseIntensity = mgf_data.flatMap(lambda x: get_parsedata(x, 'pep_intensity'))
    return parseMass.zip(parseIntensity).cache()
    # massMean = parseMass.mean()
    # massStd = parseMass.stdev()
    # intensityMean = parseIntensity.mean()
    # intensityStd = parseIntensity.stdev()
    # scaledMass = parseMass.map(lambda x: (x-massMean)/massStd)
    # scaledIntensity = parseIntensity.map(lambda x: (x-intensityMean)/intensityStd)
    # scaledData = scaledMass.zip(scaledIntensity).cache()
    # sampleMass = scaledMass.take(100)
    # sampleIntensity = scaledIntensity.take(100)

In [17]:
def error(clusters,point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

def wssse_kmeans(data, k):
    clusters = KMeans.train(data, k, maxIterations=iterations, runs=runs, initializationMode=knnmode)
    return data.map(lambda point: error(clusters,point)).reduce(lambda x, y: x + y)

In [21]:
wssse = {}
k = 1
cluster_k = range(1, k+1)
#wssse = map(lambda k: wssse_kmeans(sampleData, k, iterations, runs), cluster_k)
for k in cluster_k:
    wssse[k] = wssse_kmeans(trainData, k, iterations, runs)

TypeError: wssse_kmeans() takes exactly 2 arguments (4 given)

In [ ]:
model = KMeans.train(trainData, 23, maxIterations=iterations, runs=runs, initializationMode=knnmode)

In [ ]:
def test_knn(model):
    testData = knn_data(testPath)
    predict = testData.map(lambda x: model.predict(x))
    return predict.map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b)

testedMgf = test_knn(model).collect()

In [ ]:
testedMgf[0][1]

In [ ]:
r = range(len(model.clusterCenters))
mass_cluster = map(lambda l: model.clusterCenters[l][0], r)
intensity_cluster = map(lambda l: model.clusterCenters[l][1], r)
sizeC = map(lambda l: float(testedMgf[l][1]), r)
maxS = float(max(sizeC))
scaledSizeC = map(lambda l: (sizeC[l]/maxS)*150, r)
scaledSizeC
#clusters.clusterCenters

In [19]:
plt.scatter(mass_cluster, intensity_cluster, size=scaledSizeC)

NameError: name 'mass_cluster' is not defined